## Data Collection Code 1

In [ ]:
# Importing the necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
import os
import csv
import pandas as pd
from math import ceil

# Creates a new .csv file that the data will be written to
csv_file = open('PMscraping.csv', 'w', encoding="UTF-8", newline="")
writer = csv.writer(csv_file)

# Define the variables (future data frame columns) to be scraped
writer.writerow(['Job Title', 'location', 'Avg Salary for the job title/location', 'Summary'])

#Invoking the webdriver
locationOfWebdriver = "C:/Users/nisha/Desktop/chromedriver.exe"
driver = webdriver.Chrome(locationOfWebdriver)

# The sleep time has been set to 5 secs since loading entirely new page has proven to take longer
# than iterating through the various job postings within a single page.
page="https://www.indeed.com/jobs?q=product+manager&l=San+Francisco,+CA"
driver.get(page)
time.sleep(5)
try:
    # Find total number of product manager jobs in San Francisco Area on the portal
    # Each page defaults to showing 10 reviews, so take the ceiling of the total number of job postings divided by 10
    # to get the number of pages
    total = driver.find_element_by_xpath('//div[@id = "searchCountPages"]').text
    jobs_count = ''.join([i for i in total.split()[3] if i.isdigit()])
    n = int(ceil(float(jobs_count) / 10))
    
    # Iterate through all the pages of job posting for the product manager role in San Francisco Area
    index = 0
    k = 1  
    while index <= 10*(n-108):    # n will come out to be 157 but fetching details only for the first 50 pages so 500 records
        driver.get(page + "&start=" + str(index))
        time.sleep(2)
        print("Scraping " + "page : " + str(k))
        k = k + 1
        try:
            index = index + 10
            # Find all the job postings:
            Jobs = driver.find_elements_by_xpath('//div[@class="jobsearch-SerpJobCard unifiedRow row result clickcard"]')
            for Job in Jobs:
                # Initialize an empty dictionary for each job posting
                Jobs_dict = {}

                # Find xpaths of the fields desired as columns in future data frame
                # We use the try/except statements to account for the fact that the job postings are not required to have
                # all the fields listed below, and if a job posting does not have a certain field we wish to make the
                # corresponding field blank in that particular row, rather than quit upon receiving an error.
                try:
                    Job_Title = Job.find_element_by_xpath('.//div[@class ="title"]').text
                except:
                    Job_Title = ""
                try:
                    Location = Job.find_element_by_xpath('.//div[@class ="sjcl"]//span[@class="location accessible-contrast-color-location"]').text
                except:
                    Location = ""
                try:
                    Sal_Link = Job.find_element_by_xpath('.//div[@class ="tab-container"]//li[2]//a').get_attribute('href')
                    driver1 = webdriver.Chrome(locationOfWebdriver)
                    driver1.get(Sal_Link) #Fetching salaries from the new link obtained where the average salary for the 
                    time.sleep(2)         #same post and location has been provided
                    Avg_Sal = driver1.find_element_by_xpath('.//div[@class ="cmp-sal-salary"]').text
                    driver1.close()
                except:
                    Avg_Sal = ""
                try:
                    Summary = Job.find_element_by_xpath('.//div[@class ="summary"]').text
                except:
                    Summary = ""

                # Write the results of the above to a dictionary. Note that each overall job posting will have its
                # own dictionary, but all dictionaries for all the rows will all have the same keys. This
                # allows Selenium to write the contents of these dictionaries into a coherent .csv file
                Jobs_dict['Job Title'] = Job_Title
                Jobs_dict['Location'] = Location
                Jobs_dict['Avg_Sal'] = Avg_Sal
                Jobs_dict['Summary'] = Summary
                writer.writerow(Jobs_dict.values())

            # If an error is thrown unrelated to the above variables, print the error to the terminal
            # console, close the .csv file, and break the while loop.
        except Exception as e:
            print(e)
            csv_file.close()
            driver.close()
            break

# If an error is thrown , print the error to the terminal
# console, close the .csv file.
except Exception as e:
    print(e)
    csv_file.close()
    driver.close()
    
#Closing the csv file and the driver that has been opened
csv_file.close()
driver.close()

## Data Collection Code 2

In [ ]:
# Importing the necessary libraries
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import os
import csv
import pandas as pd
from math import ceil
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument("--test-type")
options.binary_location = "/usr/bin/chromium"

# Creates a new .csv file that the data will be written to
csv_file = open('hyderabad.csv', 'w', encoding="UTF-8", newline="")
writer = csv.writer(csv_file)

# Define the variables (future data frame columns) to be scraped
writer.writerow(['Name of the company', 'Number of Positions','Segmentation', 'Skills companies are looking for'])

#Invoking the webdriver
locationOfWebdriver = "C:/Users/nisha/Desktop/chromedriver.exe"
driver = webdriver.Chrome(locationOfWebdriver)

#Opening the naukri.com url with the help of timer and making use of sleep function
# to halt execution of next set of commands for 15 sec so that page gets loaded fully meanwhile
driver.get("https://www.naukri.com")
time.sleep(15)

#Generating handle for the later button that appears on the popup
#Clicking on the later button through the handle generated
Later_click = driver.find_element_by_xpath('//span[@id = "block"]')
Later_click.click()

#Generating handle for the search button and clicking on the same
#through the handle generated in order to enter the segment
search_job = driver.find_element_by_xpath('//span[@class = "searchJob"]')
search_job.click()
time.sleep(3)

#Searching for the position handle and the location handle
#Supplying values Information Technology and Hyderabad in Jobs and location respectively
#Clicking on the enter key in order to generate the results
search_position = driver.find_element_by_name('qp')
search_location = driver.find_element_by_name('ql')
search_position.send_keys("Information Technology")
time.sleep(2)
search_location.send_keys("Hyderabad") 
search_position.send_keys(Keys.RETURN)
time.sleep(5)

#Fetching the down arrow handle and selecting the sort by date option 
#in order to get the new job posting appear on top 
driver.find_element_by_xpath('//em[@class = "downArrow"]').click()
driver.find_element_by_xpath('/html/body/div[5]/div/div[3]/div[1]/div[1]/div/div[2]/div/div/ul/li[1]').click()
try:
    # Find total number of Information Technology jobs in Hyderabad Area on the portal(www.naukri.com)
    # Each page defaults to showing 50 reviews, so take the ceiling of the total number of job postings divided by 50
    # to get the number of pages
    total = driver.find_element_by_xpath('//div[@class = "srp_count"]//span[@class = "cnt"]').text
    jobs_count = float(str(total).split(" ")[2])
    n = int(ceil(float(jobs_count) / 50))
    print(n)
    k = 1  
    while k <= n: # Initializing k with 1 and looping it till it exhausts all the pages 
        print("Scraping " + "page : " + str(k))  # Printing the page no which is being scraped
        try:
            p = 1
            # Fetching the handle for all the job postings available on the page opened by the driver
            job_postings = driver.find_elements_by_xpath('//div[@type = "tuple"]')
            for job in job_postings: # Taking one job at a time to extract the info related to posting
                print("Scraping " + "job posting #" + str(p) + " of page : " + str(k))
                p = p + 1
                Jobs_dict = {}       # Creating an empty dictionary by the name "Jobs_dict"
                try:
                    #Fetching Company name 
                    Company = job.find_element_by_xpath('.//span[@class = "org"]').text
                except:
                    Company = ""
                try:
                    #First fetching the link where No_of_Openings and Key_skills features are embedded
                    #Opening the fetched link with the help of driver and then storing the No_of_openings
                    #to get the positions available for that role inside that company
                    Opening_link = job.find_element_by_xpath('.//span[@class ="content"]//li//a').get_attribute('href')
                    driver1 = webdriver.Chrome(locationOfWebdriver)
                    driver1.get(Opening_link)
                    time.sleep(5)
                    No_of_openings = driver1.find_element_by_xpath('//div[@class = "jd-stats"]//span[2]//span').text
                except:
                    No_of_openings = ""
                try:
                    #Segmentation is the field in which we are looking for jobs (in this case it is Information technology)
                    Segmentation = job.find_element_by_xpath('//div[@class = "small_title"]//span[@class = "frst"]').text
                except:
                    Segmentation= ""
                try:
                    #Fetching the key skills listed by the company for that particular position
                    key_skills = driver1.find_elements_by_xpath('//div//a[@class = "chip clickable"]')
                    skills_required = ','.join([key.text for key in key_skills])
                    driver1.close()
                except:
                    skills_required = "" 
                
                # Write the results of the above to a dictionary. Note that each overall job posting will have its
                # own dictionary, but all dictionaries for all the rows will all have the same keys. This
                # allows Selenium to write the contents of these dictionaries into a coherent .csv file    
                Jobs_dict['Job Title'] = Company
                Jobs_dict['No_of_openings'] = No_of_openings
                Jobs_dict['Segmentation'] = Segmentation
                Jobs_dict['skills_required'] = skills_required
                writer.writerow(Jobs_dict.values())
            
            #Fetching the Next button handle.It will be different for the first page as compared to the rest of the 
            #pages as first page will not be having previous button whereas all the other except the last page will 
            #be having the previous and next button.Last page will not be having next button for the obvious reasons.
            if(k==1):
                Next_button = driver.find_element_by_xpath('//div[@class = "pagination"]//button[@class = "grayBtn"]')
                Next_button.click()
                time.sleep(5)    
            elif(k==n):
                break
            else:
                Next_previous_button = driver.find_element_by_xpath('//div[@class = "pagination"]//a[2]//button[@class = "grayBtn"]')
                Next_previous_button.click()
                time.sleep(5)
            k = k+1
            
        # If an error is thrown unrelated to the above variables, print the error to the terminal
        # console, close the .csv file, and break the while loop.
        except Exception as e:
            print(e)
            csv_file.close()
            driver.close()
            break

# If an error is thrown unrelated to the above variables, print the error to the terminal
# console, close the .csv file, and break the while loop.           
except Exception as e:
    print(e)
    csv_file.close()
    driver.close()

#Closing the csv file and the driver that has been opened
csv_file.close()
driver.close()

## Data Collection Code 3

In [ ]:
#Importing the required libraries
from newsapi import NewsApiClient
import csv
import pandas as pd

#Supplying the api_key genrated to the NewsApiClient function and storing it in the newsapi variable
newsapi = NewsApiClient(api_key='73490089db3547b7bf4742da780c48b2')

#Printing newsapi variable
newsapi 

#Buiding the API query (topic : economy , Source : Wall Street Journal articles , data extracted for the last 6 months , 
# sorted according to the published date (newer articles will come first , older articles will appear later))
all_articles = newsapi.get_everything(q='Economy',
                                      sources='the-wall-street-journal',
                                      domains='wsj.com',
                                      from_param='2019-09-15',
                                      to='2019-10-14',
                                      language='en',
                                      sort_by='publishedAt',
                                      page=5)

#Printing all_articles variables i.e. dispalying it's contents
all_articles

# Viewing only the details related to first article
all_articles['articles'][0]

# Creates a new .csv file that the data will be written to
csv_file = open('news_api.csv', 'w', encoding="UTF-8", newline="")
writer = csv.writer(csv_file)

# Define the variables (future data frame columns) to be scraped
writer.writerow(['source_id', 'source_name', 'author_name', 'title' , 'description'])

# Iterating through all the articles (max allowed : 20 ) satisfying the filtration criteria.
# Initialize an empty dictionary for each article.
# Fetching details related to an article and writing the results of the above to a dictionary
# Note that each article will have its own dictionary, but all dictionaries for all the rows will all have the same keys. 
# This allows Selenium to write the contents of these dictionaries into a coherent .csv file
articles_dict = {}
for i in range(0,20,1):
    articles_dict['source_id'] = all_articles['articles'][i]['source']['id']
    articles_dict['source_name'] = all_articles['articles'][i]['source']['name']
    articles_dict['author_name'] = all_articles['articles'][i]['author']
    articles_dict['title'] = all_articles['articles'][i]['title']
    articles_dict['description'] = all_articles['articles'][i]['description']
    writer.writerow(articles_dict.values())

#Close the csv file 
csv_file.close()